In [1]:
import pandas as pd
import datetime as dt 

In [2]:
# sqlite3 package comes with the Python installation
import sqlite3

In [3]:
connection = sqlite3.connect('breakout.db')

cursor = connection.cursor()

In [4]:
cursor.execute('DROP TABLE IF EXISTS parents')

# 1. Simple SELECTS (on the parents table)

In [5]:
# Create the table
sql_command = '''
CREATE TABLE parents AS
  SELECT "abraham" AS parent, "barack" AS child UNION
  SELECT "abraham",           "clinton"         UNION
  SELECT "delano",            "herbert"         UNION
  SELECT "fillmore",          "abraham"         UNION
  SELECT "fillmore",          "delano"          UNION
  SELECT "fillmore",          "grover"          UNION
  SELECT "eisenhower",        "fillmore";
'''

cursor.execute(sql_command)

In [6]:
connection.commit()

In [7]:
# 1. SELECT all records in the table.
pd.read_sql_query('SELECT * FROM parents;', connection)

,parent,child
0,abraham,barack
1,abraham,clinton
2,delano,herbert
3,eisenhower,fillmore
4,fillmore,abraham
5,fillmore,delano
6,fillmore,grover


In [8]:
#2. SELECT child and parent, where abraham is the parent.
pd.read_sql_query('SELECT child,parent FROM parents WHERE parent="abraham"', connection)

,child,parent
0,barack,abraham
1,clinton,abraham


In [9]:
#3. SELECT all children that have an 'e' in their name (hint: use LIKE and '%e%').
sql_command = '''
SELECT child
FROM parents
WHERE child LIKE '%e%'
'''
pd.read_sql_query(sql_command,connection)

,child
0,herbert
1,fillmore
2,delano
3,grover


In [10]:
#4. SELECT all unique parents (use SELECT DISTINCT) and order them by name, descending order (i.e. fillmore first)
pd.read_sql_query('SELECT DISTINCT parent FROM parents ORDER BY parent DESC', connection)

,parent
0,fillmore
1,eisenhower
2,delano
3,abraham


In [11]:
#5. SELECT all dogs that are siblings (one-to-one relations). Only show a sibling pair once.
sql_command = '''
SELECT parent, COUNT(*)
FROM parents
GROUP BY parent
HAVING COUNT(*) > 1
'''
pd.read_sql_query(sql_command,connection)

,parent,COUNT(*)
0,abraham,2
1,fillmore,3


In [18]:
pd.read_sql_query('SELECT  x.child , y.child  FROM parents x, parents y WHERE x.parent=y.parent and x.child > y.child '
                  , connection)

,child,child
0,clinton,barack
1,delano,abraham
2,grover,abraham
3,grover,delano


# 2. JOINS

In [19]:
cursor.execute('DROP TABLE IF EXISTS dogs')

In [20]:
sql_command = '''
CREATE TABLE dogs AS
  SELECT "abraham" AS name, "long" AS fur UNION
  SELECT "barack",          "short"       UNION
  SELECT "clinton",         "long"        UNION
  SELECT "delano",          "long"        UNION
  SELECT "eisenhower",      "short"       UNION
  SELECT "fillmore",        "curly"       UNION
  SELECT "grover",          "short"       UNION
  SELECT "herbert",         "curly";
'''
cursor.execute(sql_command)

In [21]:
connection.commit()

In [22]:
# 1.COUNT the number of short haired dogs
pd.read_sql_query('SELECT Count(*) FROM dogs WHERE fur="short"',connection)

,Count(*)
0,3


In [23]:
#2.JOIN tables parents and dogs and SELECT the parents of curly dogs.
sql_command = '''
SELECT parent
FROM parents
JOIN dogs ON parents.child=dogs.name
WHERE dogs.fur="curly"
'''
pd.read_sql_query(sql_command,connection)

,parent
0,eisenhower
1,delano


In [24]:
#3. JOIN tables parents and dogs, and SELECT the parents and children that have the same fur type. Only show them once.
sql_command = '''
SELECT parent, child
FROM parents
JOIN dogs a ON a.name = parents.child
JOIN dogs b ON b.name = parents.parent
WHERE a.fur = b.fur
'''
pd.read_sql_query(sql_command,connection)

,parent,child
0,abraham,clinton


# 3. Aggregate functions, numerical logic and grouping

In [25]:
cursor.execute('DROP TABLE IF EXISTS animals')

In [26]:
sql_command = '''
create table animals as
 select "dog" as kind, 4 as legs, 20 as weight union
 select "cat" , 4 , 10 union
 select "ferret" , 4 , 10 union
 select "parrot" , 2 , 6 union
 select "penguin" , 2 , 10 union
select "t-rex" , 2 , 12000;
'''
cursor.execute(sql_command)
connection.commit()

In [27]:
# 1.SELECT the animal with the minimum weight. Display kind and min_weight.
sql_command = '''
SELECT kind, MIN(weight)
FROM animals;
'''
pd.read_sql_query(sql_command,connection)

,kind,MIN(weight)
0,parrot,6


In [28]:
#2. Use the aggregate function AVG to display a table with the average number of legs and the average weight.
sql_command = '''
SELECT AVG(legs),AVG(weight)
FROM animals
'''
pd.read_sql_query(sql_command,connection)

,AVG(legs),AVG(weight)
0,3.0,2009.333333


In [29]:
#3. SELECT the animal kind(s) that have more than two legs, but weighs less than 20. Display kind, weight, legs.
sql_command = '''
SELECT kind, weight, legs
FROM animals
WHERE legs>2 AND weight<20
'''
pd.read_sql_query(sql_command,connection)

,kind,weight,legs
0,cat,10,4
1,ferret,10,4


In [30]:
#4. SELECT the average weight for all the animals with 2 legs and the animals with 4 legs (by using GROUP BY).
sql_command = '''
SELECT kind,AVG(weight)
FROM animals
WHERE legs=2 OR legs=4
GROUP BY kind
'''
pd.read_sql_query(sql_command,connection)

,kind,AVG(weight)
0,cat,10.0
1,dog,20.0
2,ferret,10.0
3,parrot,6.0
4,penguin,10.0
5,t-rex,12000.0
